### Multi-Output Regressor (LSTM)

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib

from numpy import array, hstack, math
from numpy.random import uniform

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('./data/fl_2022_LastName.csv.gz', nrows = 100000)

In [3]:
# build n-gram list
NGRAMS = 2
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(df.name_last)
vocab = vect.vocabulary_
len(vocab)

993

In [4]:
import operator
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
cols = list(map(operator.itemgetter(0), sorted_vocab))

In [5]:
count_df = pd.DataFrame(a.todense(), columns=cols)
count_df

,',-,A,B,C,D,E,F,G,H,...,zp,zq,zr,zs,zt,zu,zv,zw,zy,zz
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
99996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
pd.set_option('display.max_rows', 20)
count_df.sum().sort_values(ascending=False)

Ba    21077
ar    20847
er    20668
Be    17523
an    16141
      ...  
Ef        3
qw        3
qs        3
Vr        3
 '        3
Length: 993, dtype: int64

In [7]:
# sort n-gram by freq (highest -> lowest)
words = [(a[:, c].sum(), b) for b, c in vocab.items()]
words.sort(reverse=True)

words_list = ['UNK'] + [w[1] for w in words]
num_words = len(words_list)
print(f"num_words = {num_words}")

def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    word_indices = {w: idx for idx, w in enumerate(words_list)}
    wi = [word_indices.get(''.join(i), 0) for i in a]
    return wi

# build X from index of n-gram sequence
X = np.array(df.name_last.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = [len(x) for x in X]
max_feature_len = np.max(X_len)
avg_feature_len = np.mean(X_len)

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = df.iloc[:, 1:6]

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

num_words = 994
Max feature len = 29, Avg. feature len = 9


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')

sequences = [torch.tensor(seq) for seq in X_train]
X_train = pad_sequence(sequences, batch_first=True)
sequences = [torch.tensor(seq) for seq in X_test]
X_test = pad_sequence(sequences, batch_first=True)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

input_size = X_train.shape[1]
output_size = y_train.shape[1]

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

80000 train sequences
20000 test sequences
Pad sequences (samples x time)
X_train shape: torch.Size([80000, 28])
X_test shape: torch.Size([20000, 29])
y_train shape: (80000, 5)
y_test shape: (20000, 5)


In [10]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Convert the input and output data to tensors
X_train = torch.tensor(X_train, dtype=torch.long)  # Convert to long datatype
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.long)  # Convert to long datatype
y_test = torch.tensor(y_test.values, dtype=torch.float32)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm1 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)  # Remove .long() conversion here
        out, _ = self.lstm1(embedded)
        out, _ = self.lstm2(out)
        out = self.linear(out[:, -1, :])
        return out

# Define the model parameters
input_size = int(torch.max(torch.max(X_train), torch.max(X_test))) + 1  # Adjust input_size calculation
hidden_size = 64  # Number of hidden units
output_size = y_train.shape[1]  # Number of output variables

# Create an instance of the LSTM model
model = LSTMModel(input_size, hidden_size, output_size)

# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Set the batch size and number of epochs
batch_size = 32
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    # Shuffle the training data
    indices = torch.randperm(X_train.shape[0])
    X_train_shuffled = X_train[indices]
    y_train_shuffled = y_train[indices]

    # Mini-batch gradient descent
    for i in range(0, X_train.shape[0], batch_size):
        # Get the mini-batch
        inputs = X_train_shuffled[i:i+batch_size]
        targets = y_train_shuffled[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)  # Remove unsqueeze(1) here
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the training loss for this epoch
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluation on the test set
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)  # Remove unsqueeze(1) here
    test_loss = criterion(test_outputs, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")

/tmp/ipykernel_505934/4085026627.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.long)  # Convert to long datatype
/tmp/ipykernel_505934/4085026627.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.long)  # Convert to long datatype


Epoch [1/10], Loss: 0.1216
Epoch [2/10], Loss: 0.1309
Epoch [3/10], Loss: 0.1110
Epoch [4/10], Loss: 0.1305
Epoch [5/10], Loss: 0.1039
Epoch [6/10], Loss: 0.0955
Epoch [7/10], Loss: 0.0913
Epoch [8/10], Loss: 0.0732
Epoch [9/10], Loss: 0.0658
Epoch [10/10], Loss: 0.0683
Test Loss: 0.0747


In [11]:
predictions = model(X_test)

In [23]:
(torch.argmax(y_test, dim = 1).float() == torch.argmax(predictions, dim = 1).float()).float().mean()

tensor(0.7149)

In [26]:
pd.crosstab(torch.argmax(y_test, dim = 1), torch.argmax(predictions, dim = 1))

col_0,1,2,3
row_0,,,
0,101,8,694
1,5913,38,1516
2,222,158,1656
3,587,85,8227
4,135,14,646


In [31]:
(torch.argmax(y_test, dim = 1) == 3).float().mean()

tensor(0.4450)

In [36]:
print("asian MSE: %.4f" % mean_squared_error(y_test[:, 0].detach().numpy(), predictions[:, 0].detach().numpy()))
print("hispanic MSE: %.4f" % mean_squared_error(y_test[:, 1].detach().numpy(), predictions[:, 1].detach().numpy()))
print("nh_black MSE: %.4f" % mean_squared_error(y_test[:, 2].detach().numpy(), predictions[:, 2].detach().numpy()))
print("nh_white MSE: %.4f" % mean_squared_error(y_test[:, 3].detach().numpy(), predictions[:, 3].detach().numpy()))

asian MSE: 0.0297
hispanic MSE: 0.0883
nh_black MSE: 0.0744
nh_white MSE: 0.1415


In [38]:
y_test.shape

torch.Size([20000, 5])

In [40]:
print("base asian MSE: %.4f" % mean_squared_error(y_test[:, 0], torch.mean(y_test[:, 0]).repeat(y_test.shape[0])))
print("base white MSE: %.4f" % mean_squared_error(y_test[:, 3], torch.mean(y_test[:, 3]).repeat(y_test.shape[0])))

base asian MSE: 0.0292
base white MSE: 0.2221


In [45]:
torch.save(model, 'models/fl_last_name_lstm_multioutput.pt')